In [8]:
import pandas as pd

period = '5min'
pair = 'BTC_ETH'
lag = 12

df = pd.read_csv('../datasets/{}/{}.csv'.format(period, pair), index_col=0, header=0)
df.head()

,open,high,low,close,volume
datetime,,,,,
1474205100,0.021000,0.021039,0.021000,0.021038,4.296642
1474205400,0.021042,0.021049,0.021038,0.021046,3.987229
1474205700,0.021042,0.021042,0.021000,0.021038,19.966953
1474206000,0.021025,0.021038,0.021000,0.021030,27.707384
1474206300,0.021001,0.021038,0.021001,0.021036,3.879610


In [83]:
df = pd.read_csv('../datasets/{}/{}.csv'.format(period, pair), index_col=0)
close_lag_fields = ['close-{}'.format(n+1) for n in range(lag)]
for n in range(1, lag+1):
    df['close-' + str(n)] = df['close'] - df.shift(n)['close']
df['up'] = df['close'] < df.shift(-1)['close']
df.dropna(inplace=True)

In [84]:
import numpy as np

In [85]:
X = df[close_lag_fields].values
y = np.asarray([np.eye(2)[v] for v in df['up'].astype(int).values])

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = scaler.fit_transform(X)

In [86]:
X.shape

(105108, 12)

In [102]:
y[3]

array([ 1.,  0.])

In [88]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

seed = 7

np.random.seed(seed)

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

model = Sequential()
model.add(Dense(125, input_dim=12, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [97]:
model.fit(X, y, epochs=100, batch_size=32, verbose=1)
model.save('keras_5min_btc_eth.h5')

Epoch 1/100
105108/105108 [==============================] - 4s - loss: 0.6854 - acc: 0.5438     
Epoch 2/100
105108/105108 [==============================] - 4s - loss: 0.6851 - acc: 0.5455     
Epoch 3/100
105108/105108 [==============================] - 4s - loss: 0.6846 - acc: 0.5450     
Epoch 4/100
105108/105108 [==============================] - 4s - loss: 0.6842 - acc: 0.5458     
Epoch 5/100
105108/105108 [==============================] - 4s - loss: 0.6836 - acc: 0.5473     
Epoch 6/100
105108/105108 [==============================] - 4s - loss: 0.6834 - acc: 0.5498     
Epoch 7/100
105108/105108 [==============================] - 4s - loss: 0.6829 - acc: 0.5481     
Epoch 8/100
105108/105108 [==============================] - 4s - loss: 0.6828 - acc: 0.5499     
Epoch 9/100
105108/105108 [==============================] - 4s - loss: 0.6822 - acc: 0.5520     
Epoch 10/100
105108/105108 [==============================] - 4s - loss: 0.6816 - acc: 0.5514     
Epoch 11/100
105108

105108/105108 [==============================] - 4s - loss: 0.6572 - acc: 0.5807     
Epoch 84/100
105108/105108 [==============================] - 3s - loss: 0.6565 - acc: 0.5808     
Epoch 85/100
105108/105108 [==============================] - 4s - loss: 0.6555 - acc: 0.5823     
Epoch 86/100
105108/105108 [==============================] - 4s - loss: 0.6562 - acc: 0.5815     
Epoch 87/100
105108/105108 [==============================] - 4s - loss: 0.6552 - acc: 0.5815     
Epoch 88/100
105108/105108 [==============================] - 4s - loss: 0.6548 - acc: 0.5823     
Epoch 89/100
105108/105108 [==============================] - 4s - loss: 0.6561 - acc: 0.5813     
Epoch 90/100
105108/105108 [==============================] - 4s - loss: 0.6550 - acc: 0.5828     
Epoch 91/100
105108/105108 [==============================] - 4s - loss: 0.6545 - acc: 0.5834     
Epoch 92/100
105108/105108 [==============================] - 4s - loss: 0.6547 - acc: 0.5831     
Epoch 93/100
105108/105

In [98]:
y_predict = model.predict(X)

In [99]:
pdf = pd.DataFrame(y_predict)

In [100]:
pdf.head()

,0,1
0,0.506094,0.493906
1,0.486696,0.513304
2,0.520348,0.479652
3,0.691398,0.308602
4,0.505755,0.494245
